## Retrieval

First we have to load the corpus.

In [ ]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
from datasets import load_dataset
chunk_size = 150
chunk_overlap = 50
meta_corpus = load_dataset(
    "json",
    data_files=f"data/corpus_chunks_{chunk_size}_{chunk_overlap}.jsonl",
    split="train"
).to_list()

In [ ]:
import copy
import unicodedata as ud
import re
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm
import string

def split_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.lower().split()
    words = [word for word in words if len(word.strip()) > 0]
    return words

## initiate BM25 retriever
tokenized_corpus = [split_text(doc["passage"]) for doc in tqdm(meta_corpus)]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import pickle
from pyvi.ViTokenizer import tokenize
import numpy as np
from tqdm.notebook import tqdm 

## initiate semantic rertiever
with open(f'data/corpus_embedding_w150_{chunk_size}_{chunk_overlap}.pkl', 'rb') as f:
    corpus_embs = pickle.load(f)

embedder = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

In [ ]:
print(len(meta_corpus))
print(len(corpus_embs))

To improve from simple single BM25 retriever, we now incorporate a sentence transformer model to do the semantic search. \
We will use `bkai-foundation-models/vietnamese-bi-encoder` model which supports Vietnamese pretty well.

In [ ]:
from copy import deepcopy
def retrieve(question, topk=5, score_evaluate ='combined_score', threshold_score= 0):
    """
    Get most relevant chunks to the question using combination of BM25 and semantic scores.
    """
    ## initialize query for each retriever (BM25 and semantic)
    tokenized_query = split_text(question)
    segmented_question = tokenize(question)
    question_emb_output = embedder.encode([segmented_question])
    question_emb = question_emb_output/ np.linalg.norm(question_emb_output, axis=1)[:, np.newaxis]

    ## get BM25 and semantic scores
    bm25_scores = bm25.get_scores(tokenized_query)
    semantic_scores = question_emb @ corpus_embs.T
    semantic_scores = semantic_scores[0]

    ## update chunks' scores. 
    max_bm25_score = max(bm25_scores)
    min_bm25_score = min(bm25_scores)
    def normalize(x):
        return (x - min_bm25_score + 0.1) / \
        (max_bm25_score - min_bm25_score + 0.1)
        
    corpus_size = len(meta_corpus)
    for i in range(corpus_size):
        meta_corpus[i]["bm25_score"] = bm25_scores[i]
        meta_corpus[i]["bm25_normed_score"] = normalize(bm25_scores[i])
        meta_corpus[i]["semantic_score"] = semantic_scores[i]
        
        meta_corpus[i]['cosine_score'] = cosine(question_emb_output[0],corpus_embs[i])

    ## compute combined score (BM25 + semantic)
    for passage in meta_corpus:
        passage["combined_score"] = passage["bm25_normed_score"] * 0.4 + \
                                    passage["semantic_score"] * 0.6
        
    ## sort passages by the combined score

    sorted_passages = sorted(meta_corpus, key=lambda x: x[score_evaluate], reverse=True)
    sorted_passages = [each for each in sorted_passages if each[score_evaluate]>threshold_score]
    return sorted_passages[:topk]

In [ ]:
retrieve("Mía có nguồn gốc ở đâu", topk=20, score_evaluate='combined_score')

In [ ]:
df_evalutate =  pd.read_excel('evaluat_dataset_2.xlsx' ).drop(['Unnamed: 0'], axis=1)[['question', 'answer', 'source_doc']]

In [ ]:
df_evalutate['cosine_doc'] = None
df_evalutate['combined_doc'] = None

df_evalutate['cosine_score_results'] = None
df_evalutate['combined_score_results'] = None
topk = 3
thresh_cosine = 0
thresh_combined = 0.5
for index_evaluate in tqdm(range(len(df_evalutate))):
    test_doc = df_evalutate.iloc[index_evaluate]
    question = test_doc['question']

    # result_retrieve_cosine = retrieve(question, topk=topk, score_evaluate='cosine_score', threshold_score = thresh_cosine)
    # consine_context = ""
    # cosine_title = []
    # for index in range(len(result_retrieve_cosine)):
    #     doc_retrieved = result_retrieve_cosine[index]
    #     context_doc = f"Context [{index}]: {doc_retrieved['passage']}\nCosine_score: {doc_retrieved['cosine_score']}   Combined_score: {doc_retrieved['combined_score']}\n"
    #     consine_context += context_doc
    #     cosine_title.append(doc_retrieved['title'])

    result_retrieve_combined = retrieve(question, topk=topk, score_evaluate='combined_score', threshold_score = thresh_combined)
    combined_context = ''
    combine_title = []
    for index in range(len(result_retrieve_combined)):
        doc_retrieved = result_retrieve_combined[index]
        context_doc = f"Context [{index}]: {doc_retrieved['passage']}\nCosine_score: {doc_retrieved['cosine_score']}   Combined_score: {doc_retrieved['combined_score']}\n"
        combined_context += context_doc
        combine_title.append(doc_retrieved['title'])
    
    # df_evalutate.at[index_evaluate, 'cosine_doc'] = cosine_title
    df_evalutate.at[index_evaluate, 'combined_doc'] = combine_title


    # df_evalutate.at[index_evaluate, 'cosine_score_results'] = consine_context
    df_evalutate.at[index_evaluate, 'combined_score_results'] = combined_context

df_evalutate[['question', 'answer', 'source_doc',  'combined_doc',  'combined_score_results']].to_excel(f'{chunk_size}_{chunk_overlap}_result_evaluate_top{topk}_threshold_{thresh_combined}.xlsx')

# df_evalutate[['question', 'answer', 'source_doc', 'cosine_doc', 'combined_doc', 'cosine_score_results', 'combined_score_results']].to_excel(f'{chunk_size}_{chunk_overlap}_result_evaluate_top{topk}.xlsx')
    

In [ ]:
df_evalutate.head(2)

## Smoothing contexts

Using combination of BM25 and semantic score may still not yield the best result (because of the reasons you will see at the end of this section) \
We can do better by applying several techniques to form suitable contexts to feed to the LLM. \
The following blocs will implement some utility functions with such techniques.

In [ ]:
from copy import deepcopy
from underthesea import sent_tokenize

# GET DOCUMENT IS consecutive
def extract_consecutive_subarray(numbers):
    subarrays = []
    current_subarray = []
    for num in numbers:
        if not current_subarray or num == current_subarray[-1] + 1:
            current_subarray.append(num)
        else:
            subarrays.append(current_subarray)
            current_subarray = [num]

    subarrays.append(current_subarray)  # Append the last subarray
    return subarrays

# MERGE CONTEXT IF THEY ARE THE SERIAL CONTEXT
def merge_contexts(passages):
    passages_sorted_by_id = sorted(passages, key=lambda x: x["id"], reverse=False)
    # psg_texts = [x["passage"].strip("Title: ").strip(x["title"]).strip() 
    #              for x in passages_sorted_by_id]
    # print(passages_sorted_by_id)
    if not len(passages_sorted_by_id):
        return []
    psg_ids = [x["id"] for x in passages_sorted_by_id]
    consecutive_ids = extract_consecutive_subarray(psg_ids)
    merged_contexts = []
    consecutive_psgs = []
    b = 0
    for ids in consecutive_ids:
        psgs = passages_sorted_by_id[b:b+len(ids)]
        psg_texts = [x["passage"].strip("Title: ").strip(x["title"]).strip() 
                     for x in psgs]
        merged = f"Title: {psgs[0]['title']}\n\n" + " ".join(psg_texts)
        b = b+len(ids)
        merged_contexts.append(dict(
            title=psgs[0]['title'], 
            passage=merged,
            score=max([x["combined_score"] for x in psgs]),
            merged_from_ids=ids
        ))
    return merged_contexts

def discard_contexts(passages):
    sorted_passages = sorted(passages, key=lambda x: x["score"], reverse=False)
    if len(sorted_passages) == 1:
        return sorted_passages
    else:
        shortened = deepcopy(sorted_passages)
        for i in range(len(sorted_passages) - 1):
            current, next = sorted_passages[i], sorted_passages[i+1]
            if next["score"] - current["score"] >= 0.05:
                shortened = sorted_passages[i+1:]
        return shortened

def expand_context(passage, word_window=60, n_sent=3):
    # psg_id = passage["id"]
    merged_from_ids = passage["merged_from_ids"]
    title = passage["title"]
    prev_id = merged_from_ids[0] - 1
    next_id = merged_from_ids[-1] + 1
    strip_title = lambda x: x["passage"].strip(f"Title: {x['title']}\n\n")
    
    texts = []
    if prev_id in range(0, len(meta_corpus)):
        prev_psg = meta_corpus[prev_id]
        if prev_psg["title"] == title: 
            prev_text = strip_title(prev_psg)
            # prev_text = " ".join(prev_text.split()[-word_window:])
            prev_text = " ".join(sent_tokenize(prev_text)[-n_sent:])
            texts.append(prev_text)
            
    texts.append(strip_title(passage))
    
    if next_id in range(0, len(meta_corpus)):
        next_psg = meta_corpus[next_id]
        if next_psg["title"] == title: 
            next_text = strip_title(next_psg)
            # next_text = " ".join(next_text.split()[:word_window])
            next_text = " ".join(sent_tokenize(next_text)[:n_sent])
            texts.append(next_text)

    expanded_text = " ".join(texts)
    expanded_text = f"{expanded_text}"
    new_passage = deepcopy(passage)
    new_passage["passage"] = expanded_text
    return new_passage

def expand_contexts(passages, word_window=60, n_sent=3):
    new_passages = [expand_context(passage) for passage in passages]
    return new_passages
    
def collapse(passages):
    new_passages = deepcopy(passages)
    titles = {}
    for passage in new_passages:
        title = passage["title"]
        if not titles.get(title):
            titles[title] = [passage]
        else:
            titles[title].append(passage)
    best_passages = []
    for k, v in titles.items():
        best_passage = max(v, key= lambda x: x["score"])
        best_passages.append(best_passage)
    return best_passages

Note that, with our current chunking strategy, each chunk is a passage of exact 150 words (separated by space), not a comprehensive paragraph. The following function will transform retrieved chunks into whole paragraphs. This function also does some heuristics to expand the context window and discard seem-to-be irrelevant contexts.

In [ ]:
def smooth_contexts(passages):
    """Make the context fed to the LLM better.
    Args:
        passages (list): Chunks retrieved from BM25 + semantic retrieval. 
        
    Returns:
        list: List of whole paragraphs, usually will be more relevant to the initital question.
    """
    # 1. If consecutive chunks are rertieved, merge them into one big chunk to ensure the continuity.
    merged_contexts = merge_contexts(passages)
    # 2. A heuristic to discard irrelevevant contexts. 
    # It seems to be better to only keep what are elevant so that the model can focus.
    # Also this reduce #tokens LLM has to read.
    shortlisted_contexts = discard_contexts(merged_contexts)
    # 3. Another heuristic. this step is to take advantage of long context understanding of the LLM.
    # In many cases, the retrieved passages are just consecutive words, not a comprehensive paragraph.
    # This is to expand the passage to the whole paragraph that surrounds it. 
    # My intuition about this is that whole paragraph will add necessary and relevant information.
    expanded_contexts = expand_contexts(shortlisted_contexts)
    # 4. Now after all the merging and expanding, if what are left for us is more than one paragraphs
    # from the same wiki page, then we will only take paragraph with highest retrieval score.
    collapsed_contexts = collapse(expanded_contexts)
    return collapsed_contexts

In [ ]:
## I encourage you to investigate each smoothing step using the following example
## to understand the benefit of them. 
## You will see that after each step, we will obtain "better" contexts.

question = "Các biện pháp phòng trừ bệnh khô cành khô quả cho cây cà phê?"
top_passages = retrieve(question, topk=3)
merged_contexts = merge_contexts(top_passages)
shortlisted_contexts = discard_contexts(merged_contexts)
expanded_contexts = expand_contexts(shortlisted_contexts)
collapsed_contexts = collapse(expanded_contexts)

In [ ]:
## Uncomment each of these variable to see the differences.

# top_passages
# merged_contexts
# shortlisted_contexts
# expanded_contexts
# collapsed_contexts

## Generate

In [ ]:
from transformers import GenerationConfig, TextStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import os
os.environ['HF_HOME'] = "/media/user123456/D2/tmp/cache/huggingface"
# os.environ['HUGGINGFACE_HUB_CACHE'] = "/media/user123456/D2/tmp/cache/huggingface/hub"
torch_dtype = torch.bfloat16
model_id = "Viet-Mistral/Vistral-7B-Chat"
device = "cuda:1"
cache_dir = "/media/user123456/D2/tmp/cache/huggingface/hub"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_cache=True,
                                           cache_dir=cache_dir
                                        )
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # change to torch.float16 if you're using V100
    # device_map="auto",
    use_cache=True,
    cache_dir=cache_dir
).eval().to(device)

# model = model.eval().to(device)

In [ ]:
def generate(prompt, max_new_tokens=1024, tempearature = 0, top_p = 0):
    """Text completion with a given prompt. In other words, give answer to your question.
    Args:
        prompt (str): Basically <instruction> + <question> + <retrieved_context>
        generation_config (not existed yet): For now, please manually tweak hyperparameters
        for generation in the `generation_config` below. Uncomment necessary arguments as you wish.
    Returns:
        list: an answer to the question within the prompt.
    """
    device = "cuda:1"
    # print(device)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)
    model.eval()
    with torch.no_grad():
        generation_config = GenerationConfig(
            repetition_penalty=1.13,
            max_new_tokens=max_new_tokens,
            temperature=tempearature,
            top_p=top_p,
            # top_k=20,
            # bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # eos_token_id=0, # for open-end generation.
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True,
            return_dict_in_generate=True,
            output_attentions=False,
            output_hidden_states=False,
            output_scores=False,
        )
        streamer = TextStreamer(tokenizer, skip_prompt=True)
        generated = model.generate(
            inputs=input_ids,
            generation_config=generation_config,
            streamer=streamer,
        )

    gen_tokens = generated["sequences"].cpu()[:, len(input_ids[0]):].to(device)
    output = tokenizer.batch_decode(gen_tokens)[0]
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()

In [ ]:
prompt_template = (
"<s>[INST] <<SYS>>\n"
"<</SYS>> \n\n"
"""
<context>
{context}
</context>
Question: {question} [/INST]"""
)

def get_prompt(question, contexts, prompt_template=prompt_template):
    context = "\n\n".join([f" {x['passage']}" for i, x in enumerate(contexts)])
    # instruction = 'As an intelligent AI model, your task is to analyze and integrate information from multiple contexts given below in order to answer questions and provide citations.'
    prompt = prompt_template.format(
        # instruction=instruction,
        context=context,
        question = question,
        output=''
    )
    return prompt

In [ ]:
# prompt_template = (
# "<s>[INST] <<SYS>>\n"
# "<</SYS>> \n\n"
# "{input} [/INST]"
# )

# def get_prompt(question, contexts, prompt_template=prompt_template):
#     context = "\n\n".join([f" {x['passage']}" for i, x in enumerate(contexts)])
#     # instruction = 'As an intelligent AI model, your task is to analyze and integrate information from multiple contexts given below in order to answer questions and provide citations.'

#     input = f"""
#     <context>
#     {context}
#     </context>
#     Question: {question}
#     """
#     prompt = prompt_template.format(
#         # instruction=instruction,
#         input=input,
#         output=''
#     )
#     return prompt

In [ ]:
question = "Thủ đô của Việt Nam"
get_prompt(question,"")

### PROMPT SAMPLE 1:

In [ ]:
prompt_template_1 = (
"<s>[INST] <<SYS>>\n"
"Bạn là một trợ lí Tiếng Việt trả lời câu hỏi nhiệt tình và trung thực chỉ dựa vào thông tin được cho.\n"
"Bạn chỉ dựa vào nội dung của [DOCUMENT] được cung cấp để trả lời [QUESTION]. Đừng thêm bất kỳ thông tin cá nhân hoặc thông tin từ bên ngoài vào câu trả lời của bạn.\n"
"Bạn trả lời đúng trọng tâm của câu hỏi và tránh trả lời quá dài dòng hoặc không liên quan.\n"
# "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực. Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác."
"""Hãy trả về kết quả là "Tôi không biết trả lời câu hỏi này" nếu bạn không chắc chắn về đáp án của mình.\n"""
"<</SYS>> \n\n"
"""
<context>
{context}
</context>
Question: {question} [/INST]"""
)


In [ ]:
print(prompt_template_1)

### PROMPT SAMPLE 2:

In [ ]:
prompt_template_2 = (
"<s>[INST] <<SYS>>\n"
"Bạn là một trợ lí Tiếng Việt trả lời câu hỏi nhiệt tình và trung thực.\n"
"Nhiệm vụ chính của bạn là trả lời câu hỏi chỉ dựa vào đoạn thông tin được đưa ra.\n"
"Giữ cái câu trả lời ngắn gọn và súc tích.\n"
# "Câu trả lời của bạn đúng trọng tâm của câu hỏi và tránh trả lời quá dài dòng hoặc không liên quan.\n"
# "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực. Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác."
"""Hãy trả về kết quả là "Tôi không biết trả lời câu hỏi này" nếu bạn không chắc chắn về câu trả lời của mình.\n"""
"""Hãy trả về kết quả là "Tôi không biết trả lời câu hỏi này" nếu đoạn thông tin được đưa ra không có dữ liệu liên quan đến câu hỏi.\n"""
"<</SYS>> \n\n"
"""
<context>
{context}
</context>
Question: {question} [/INST]"""
)


In [46]:
top_passages

[]

In [56]:
## Let's see how a prompt fed to the LLM looks like
question = "Thủ đô của Hàn Quốc"
thresh_combined= 0.65
top_p = 0
score_evaluate = 'combined_score'
top_passages = retrieve(question, topk=5,score_evaluate=score_evaluate, threshold_score = thresh_combined)
# smoothed_contexts = smooth_contexts(top_passages)
prompt = get_prompt(question, top_passages, prompt_template_3)
output = generate(prompt, tempearature=0, top_p= top_p)
print(prompt)
# print(output)

 <answer>Thủ đô của Hàn Quốc là Seoul.</answer> </s>
<s>[INST] <<SYS>>
Bạn là một trợ lí Tiếng Việt trả lời câu hỏi nhiệt tình và trung thực.
Bạn sẽ chỉ trả lời các câu hỏi của người dùng bằng thông tin được cung cấp. Bạn sẽ tuân thủ các quy tắc sau:
- Bạn không bao giờ nói dối hay bịa ra những câu trả lời không được nêu rõ ràng trong thông tin được cung cấp.
- Nếu bạn không chắc chắn về câu trả lời hoặc câu hỏi không có rõ ràng trong thông tin được cung cấp, bạn sẽ nói: "Tôi xin lỗi, tôi không biết phải trợ giúp điều đó như thế nào.".
- Bạn luôn giữ câu trả lời ngắn gọn, phù hợp và súc tích.<</SYS>>  


<context>

</context>
<question>: Thủ đô của Hàn Quốc </question> [/INST]


In [ ]:
output = generate(prompt)

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
# system_prompt += "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực."
# system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch."

# tokenizer = AutoTokenizer.from_pretrained('Viet-Mistral/Vistral-7B-Chat')
# model = AutoModelForCausalLM.from_pretrained(
#     'Viet-Mistral/Vistral-7B-Chat',
#     torch_dtype=torch.bfloat16, # change to torch.float16 if you're using V100
#     device_map="auto",
#     use_cache=True,
# )

# conversation = [{"role": "system", "content": system_prompt }]
# def generate(prompt, max_new_tokens=1024):
#     human = input("Human: ")
#     if human.lower() == "reset":
#         conversation = [{"role": "system", "content": system_prompt }]
#         print("The chat history has been cleared!")
#         continue

#     conversation.append({"role": "user", "content": human })
#     input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)
    
#     out_ids = model.generate(
#         input_ids=input_ids,
#         max_new_tokens=768,
#         do_sample=True,
#         top_p=0.95,
#         top_k=40,
#         temperature=0.1,
#         repetition_penalty=1.05,
#     )
#     assistant = tokenizer.batch_decode(out_ids[:, input_ids.size(1): ], skip_special_tokens=True)[0].strip()
#     print("Assistant: ", assistant) 
#     conversation.append({"role": "assistant", "content": assistant })

In [ ]:
## Let's test what the LLM would generate given a question and its context via a prompt.
output = generate(prompt)

Not the best but pretty accurate.

## End-to-End RAG

It's almost done. Now let's try a simple RAG pipeline with our Wikipedia corpus.

In [67]:

def rag(question, topk=3, score_evaluate='combined_score', threshold_score = 0.6, top_p = 0, prompt_template = prompt_template):
    top_passages = retrieve(question, topk=topk,score_evaluate=score_evaluate, threshold_score = threshold_score)
    if not len(top_passages):
        print("Cannot find document revelance")
        return {
        "retrieved_context": None,
        "generated_answer": "Không thể trả lời câu hỏi vì không tìm thấy thông tin liên quan trong kho dữ liệu"
    }
    smoothed_contexts = smooth_contexts(top_passages)
    # print("_____")
    # print(smoothed_contexts)
    retrieved_context = "\n\n".join([f"Context [{i+1}]: {x['passage']}" 
                    for i, x in enumerate(smoothed_contexts)])
    prompt = get_prompt(question, smoothed_contexts, prompt_template)
    output = generate(prompt, tempearature=0, top_p= top_p)
    result = {
        "retrieved_context": retrieved_context,
        "generated_answer": output.replace("</s>", "")
    }
    return result

In [68]:
thresh_combined= 0.6
top_p = 0
score_evaluate = 'combined_score'
topk = 5
questions = [
    "Cầu thủ nào dành nhiều quả bóng vàng nhất?",
    "Thành phố nào là thủ đô của Việt Nam?",
    "Công dụng của cây nha đam đối với sức khỏe?",
    "Khi nào là thời điểm tốt nhất để thực hiện kỹ thuật cắt tỉa và tạo tán cho cây cà phê?",
    "Tại sao 2Nông hỗ trợ truy xuất nguồn gốc sản phẩm?"
]
for question in questions:
    print(f"Câu hỏi: {question}")
    output = rag(question, topk=topk, score_evaluate=score_evaluate, threshold_score=thresh_combined, top_p=top_p, prompt_template =prompt_template_2)
    # print(output)
    print("---" * 30)

Câu hỏi: Cầu thủ nào dành nhiều quả bóng vàng nhất?
Cannot find document revelance
------------------------------------------------------------------------------------------
Câu hỏi: Thành phố nào là thủ đô của Việt Nam?
Cannot find document revelance
------------------------------------------------------------------------------------------
Câu hỏi: Công dụng của cây nha đam đối với sức khỏe?
 Công dụng của cây nha đam bao gồm sát khuẩn, chống lão hóa, giảm sưng, viêm, bảo vệ đường ruột, dạ dày, cải thiện chất lượng giấc ngủ và hỗ trợ não bộ. </s>
------------------------------------------------------------------------------------------
Câu hỏi: Khi nào là thời điểm tốt nhất để thực hiện kỹ thuật cắt tỉa và tạo tán cho cây cà phê?
 Thời điểm tốt nhất để thực hiện kỹ thuật cắt tỉa và tạo tán cho cây cà phê là trước khi tiến hành bón phân và trước khi mùa mưa bắt đầu. </s>
------------------------------------------------------------------------------------------
Câu hỏi: Tại sao 2Nông hỗ

In [ ]:
import pandas as pd
# test_df = pd.read_csv("/media/user123456/D2/chatbot/langchain_rag/scripts/evaluate/embeddings_models/vinai__phobert-base-v2_vncore_segment/20_vinai__phobert-base-v2_vncore_segment_256_20_cosine.csv", sep="|")
test_evaluate = pd.read_excel('evaluat_dataset_3.xlsx' ).drop(['Unnamed: 0'], axis=1)[['question', 'answer', 'source_doc']]

### PROMPT TEMPLATE 1:

In [ ]:
prompt_template_1 = (
"<s>[INST] <<SYS>>\n"
"Tôi là  một nhân viên hỗ trợ khách hàng rất tốt bụng và nhiệt tình, thích giúp đỡ khách hàng.\n"
"Tôi sẽ chỉ trả lời các câu hỏi của người dùng bằng <context> được cung cấp. Tôi sẽ tuân thủ các quy tắc sau:\n"
"- Tôi là một người tốt bụng và hữu ích, là nhân viên hỗ trợ khách hàng tốt nhất hiện nay\n"
"- Tôi không bao giờ nói dối hay bịa ra những câu trả lời không được nêu rõ ràng trong <context>\n"
"""- Nếu tôi không chắc chắn về câu trả lời hoặc câu trả lời không có rõ ràng trong <context>, tôi sẽ nói: "Tôi xin lỗi, tôi không biết phải trợ giúp điều đó như thế nào.\n" """
"- Tôi luôn giữ câu trả lời ngắn gọn, phù hợp và súc tích."
"<</SYS>> \n\n"
"""
<context>
{context}
</context>
Question: {question} [/INST]"""
)


### PROMPT TEMPLATE 2:

In [ ]:
prompt_template_2 = (
"<s>[INST] <<SYS>>\n"
"Bạn là một trợ lí Tiếng Việt trả lời câu hỏi nhiệt tình và trung thực.\n"
"Nhiệm vụ chính của bạn là trả lời câu hỏi chỉ dựa vào đoạn thông tin được đưa ra.\n"
"Giữ cái câu trả lời ngắn gọn và súc tích.\n"
# "Câu trả lời của bạn đúng trọng tâm của câu hỏi và tránh trả lời quá dài dòng hoặc không liên quan.\n"
# "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực. Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác."
"""Hãy trả về kết quả là "Tôi không biết trả lời câu hỏi này" nếu bạn không chắc chắn về câu trả lời của mình.\n"""
"""Hãy trả về kết quả là "Tôi không biết trả lời câu hỏi này" nếu đoạn thông tin được đưa ra không có dữ liệu liên quan đến câu hỏi.\n"""
"<</SYS>> \n\n"
"""
<context>
{context}
</context>
Question: {question} [/INST]"""
)


### PROMPT TEMPLATE 3:

In [ ]:
prompt_template_3 = (
"<s>[INST] <<SYS>>\n"
"Bạn là một trợ lí Tiếng Việt trả lời câu hỏi nhiệt tình và trung thực.\n"
# "Bạn sẽ trả lời các câu hỏi chỉ bằng cách sử dụng thông tin được cung cấp và tuân theo các quy tắc.\n"
# """[QUY TẮC]:"""
"""Bạn sẽ chỉ trả lời các câu hỏi của người dùng bằng thông tin được cung cấp. Bạn sẽ tuân thủ các quy tắc sau:
- Bạn không bao giờ nói dối hay bịa ra những câu trả lời không được nêu rõ ràng trong thông tin được cung cấp.
- Nếu bạn không chắc chắn về câu trả lời hoặc câu hỏi không có rõ ràng trong thông tin được cung cấp, bạn sẽ nói: "Tôi xin lỗi, tôi không biết phải trợ giúp điều đó như thế nào.".
- Bạn luôn giữ câu trả lời ngắn gọn, phù hợp và súc tích."""
"<</SYS>>  \n\n" 
"""
<context>
{context}
</context>
<question>: {question} </question> [/INST]"""
)


## EXECUTE

In [69]:
folder_save_evaluate = "./evaluate/"
prompt_name = "prompt_template_3_cosine_evaluate"    
prompt_template_use = prompt_template_3
folder_save_result_prompt = folder_save_evaluate + prompt_name + "/"
os.makedirs(folder_save_result_prompt, exist_ok=True)
with open(folder_save_result_prompt + "prompt_template.txt", 'w') as f:
    f.write(prompt_template_use)
tempearature = 0
top_p = 0
score_evaluate = 'cosine_score'
# topk = 5
for topk in [5]:
    for thresh_combined in [0.5, 0.6, 0.65, 0.7, 0.75, 0.8]:
        print(f"Top k: {topk}  ----- Threshold: {thresh_combined}")
        test_df = test_evaluate.copy()
        test_df['vietmistal_result'] = test_df['question'].apply(lambda x: rag(x, topk=topk, score_evaluate=score_evaluate, threshold_score=thresh_combined, top_p=top_p,prompt_template=prompt_template_use))
        test_df= test_df.join(pd.DataFrame(test_df['vietmistal_result'].values.tolist()))
        test_df.to_excel(folder_save_result_prompt + f"vietmistal_result_threshole_{thresh_combined}_tempearature_{tempearature}_top{topk}.xlsx", index=False)
        print("________"*10)

Top k: 5  ----- Threshold: 0.5


/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

 <answer>Nha đam có tác dụng sát khuẩn, chống lão hóa, giảm sưng, viêm,... nên được ứng dụng rộng rãi trong các phương pháp chăm sóc và làm đẹp da. Ngoài ra chúng còn có khả năng bảo vệ đường ruột, dạ dày.</answer> </s>
 Để chăm sóc đúng cách cho cây lan Vanda, hãy đảm bảo rằng chúng nhận được nhiệt độ từ 25 - 30 độ C, độ ẩm khoảng 70-80%. Tưới nước đều đặn, đặc biệt là trong thời kỳ khô hạn. Vì đây là giống ưa sáng nên chúng cần ánh sáng mạnh mẽ để nở hoa. Ngoài ra, duy trì hệ thống thoát nước thích hợp và bón phân cân đối mỗi tuần một lần cũng góp phần tạo môi trường lý tưởng cho cây phát triển khỏe mạnh. </s>
 Để phòng trừ côn trùng và nấm bệnh trên cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây, ảnh hưởng trực tiếp đến chất lượng và khả năng đậu trái. Do đó, nên tập trung cẩn thận vào các biện pháp này để đảm bảo kết quả tốt

/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

 Cây nha đam có nhiều lợi ích cho sức khỏe do tính chất kháng khuẩn và chữa bệnh tự nhiên của nó. Nó thường được sử dụng trong các sản phẩm chăm sóc da vì đặc tính dưỡng ẩm và phục hồi của nó. Nha đam cũng đã được chứng minh là hữu ích trong việc thúc đẩy tiêu hóa lành mạnh, hỗ trợ hệ thống miễn dịch và cải thiện chức năng gan. Uống nước ép lô hội tươi có thể tăng cường mức năng lượng và thậm chí giúp kiểm soát cân nặng. Tuy nhiên, quan trọng nhất là tham khảo ý kiến bác sĩ trước khi bắt đầu bất kỳ chế độ ăn uống mới hoặc thay đổi lối sống nào. </s>
 Để chăm sóc đúng cách cho cây lan Vanda, hãy đảm bảo rằng chúng nhận được nhiệt độ từ 25 - 30 độ C, độ ẩm khoảng 70-80%. Tưới nước đều đặn, đặc biệt là trong thời kỳ khô hạn. Vì đây là giống ưa sáng nên chúng cần ánh sáng mạnh mẽ để nở hoa. Ngoài ra, duy trì hệ thống thoát nước thích hợp và bón phân cân đối mỗi tuần một lần cũng góp phần tạo môi trường lý tưởng cho cây phát triển khỏe mạnh. </s>
 Để phòng trừ côn trùng và nấm bệnh trên cây

...

 Để chăm sóc đúng cách cho cây lan Vanda, hãy đảm bảo rằng nó nhận đủ lượng ánh sáng (ít nhất sáu tiếng mỗi ngày), duy trì nhiệt độ từ 25 - 30C và độ ẩm khoảng 70-80%. Tưới nước thường xuyên vào mùa hè nhưng giảm tần suất này xuống hai đến ba lần mỗi tuần trong thời kỳ nghỉ đông. Ngoài ra, bón phân cân đối hàng tháng trong suốt cả năm trừ giai đoạn nghỉ ngơi. </s>
 Để phòng trừ côn trùng và nấm bệnh trên cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây, ảnh hưởng trực tiếp đến chất lượng và khả năng đậu trái. Do đó, nên tập trung cẩn thận vào các biện pháp này để đảm bảo kết quả tốt nhất cho vườn cây ăn quả của bạn. </s>
 Để trồng và chăm sóc cây chu đinh lan, hãy làm theo hướng dẫn sau đây :

1. Chọn loại đất thích hợp: Chu đinh lan ưa đất tơi xốp, thoát nước tốt và duy trì độ ẩm khoảng 60-70 %. Đảm bảo rằng đất có hệ thống thoát

/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

phòng trừ côn trùng và nấm bệnh trên cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây, ảnh hưởng trực tiếp đến chất lượng và khả năng đậu trái. Do đó, nên tập trung cẩn thận vào các biện pháp này để đảm bảo kết quả tốt nhất cho vườn cây ăn quả của bạn. </s>
 Để trồng và chăm sóc cây lan chu đinh, hãy làm theo hướng dẫn dưới đây:
1. Chọn loại đất thích hợp: Cây lan chu đinh phát triển mạnh ở nhiều loại đất khác nhau, nhưng lý tưởng nhất là đất thịt, đất mùn, đất cát pha. Đảm bảo rằng đất tơi xốp, thoát nước tốt và giữ ẩm tốt.
2. Trồng cây đúng cách: Khi trồng cây lan chu đinh, đào hố sâu hơn rễ khoảng 5 cm. Đặt cây vào lỗ sao cho cổ rễ ngang với mặt đất. Lấp đất lên gốc cây và nén nhẹ xuống. Tưới nước ngay lập tức để đảm bảo độ ẩm cho cây.
3. Chăm sóc thường xuyên: Duy trì độ ẩm đều đặn cho cây lan chu đinh bằng cách tưới nước thườ

/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

phòng trừ côn trùng và nấm bệnh trên cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây, ảnh hưởng trực tiếp đến chất lượng và khả năng đậu trái. Do đó, nên tập trung cẩn thận vào các biện pháp này để đảm bảo kết quả tốt nhất cho vườn cây ăn quả của bạn. </s>
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
 Để chăm sóc cây sau khi thu hoạch sầu riêng, bà con cần thực hiện vệ sinh gốc cây, tỉa cành và bón phân cân đối. Điều này đảm bảo rằng cây khỏe mạnh và sẵn sàng cho mùa vụ tiếp theo. </s>
Cannot find document re

In [70]:
folder_save_evaluate = "./evaluate/"
prompt_name = "prompt_template_2_cosine_evaluate"    
prompt_template_use = prompt_template_2
folder_save_result_prompt = folder_save_evaluate + prompt_name + "/"
os.makedirs(folder_save_result_prompt, exist_ok=True)
with open(folder_save_result_prompt + "prompt_template.txt", 'w') as f:
    f.write(prompt_template_use)
tempearature = 0
top_p = 0
score_evaluate = 'cosine_score'
# topk = 5
for topk in [5]:
    for thresh_combined in [0.5, 0.6, 0.65, 0.7, 0.75, 0.8]:
        print(f"Top k: {topk}  ----- Threshold: {thresh_combined}")
        test_df = test_evaluate.copy()
        test_df['vietmistal_result'] = test_df['question'].apply(lambda x: rag(x, topk=topk, score_evaluate=score_evaluate, threshold_score=thresh_combined, top_p=top_p,prompt_template=prompt_template_use))
        test_df= test_df.join(pd.DataFrame(test_df['vietmistal_result'].values.tolist()))
        test_df.to_excel(folder_save_result_prompt + f"vietmistal_result_threshole_{thresh_combined}_tempearature_{tempearature}_top{topk}.xlsx", index=False)
        print("________"*10)

Top k: 5  ----- Threshold: 0.5


/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

 Công dụng của cây nha đam bao gồm sát khuẩn, chống lão hóa, giảm sưng, viêm, bảo vệ đường ruột, dạ dày, cải thiện chất lượng giấc ngủ và hỗ trợ não bộ. </s>
 Để chăm sóc đúng cách cây lan Vanda, hãy đảm bảo rằng nó nhận được nhiệt độ từ 25 - 30 độ C, duy trì độ ẩm khoảng 70-80%. Tưới nước đều đặn, đặc biệt là trong thời kỳ khô hạn hoặc mùa hè. Cây cũng thích ánh sáng mạnh, vì vậy đặt chúng dưới ánh mặt trời trực tiếp hoặc sử dụng bóng đèn chuyên dùng cho cây cảnh. Ngoài ra, bón phân cân đối mỗi tuần một lần trong suốt cả năm để thúc đẩy sự phát triển khỏe mạnh. </s>
 Để phòng trừ côn trùng và nấm bệnh cho cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây. </s>
 Để trồng và chăm sóc cây chu đinh lan, hãy làm theo những bước sau:

1. Chọn loại đất phù hợp: Sử dụng đất thịt, đất mùn hoặc đất cát pha với tính thấm nước cao và giữ ẩm t

/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

Cây nha đam có nhiều lợi ích cho sức khỏe như sát khuẩn, chống lão hóa, giảm sưng, viêm, bảo vệ đường ruột, dạ dày, giúp cải thiện chất lượng giấc ngủ và hỗ trợ chức năng não bộ. </s>
 Để chăm sóc đúng cách cây lan Vanda, hãy đảm bảo rằng nó nhận được nhiệt độ từ 25 - 30 độ C, duy trì độ ẩm khoảng 70-80%. Tưới nước đều đặn, đặc biệt là trong thời kỳ khô hạn hoặc mùa hè. Cây cũng thích ánh sáng mạnh, vì vậy đặt chúng dưới ánh mặt trời trực tiếp hoặc sử dụng bóng đèn chuyên dùng cho cây cảnh. Ngoài ra, bón phân cân đối mỗi tuần một lần trong suốt cả năm để thúc đẩy sự phát triển khỏe mạnh. </s>
 Để phòng trừ côn trùng và nấm bệnh cho cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây. </s>
 Để trồng và chăm sóc cây chu đinh lan, hãy làm theo những bước sau:

1. Chọn loại đất phù hợp: Sử dụng đất thịt, đất mùn hoặc đất cát pha với tính

...

 Để chăm sóc đúng cách cây lan Vanda, hãy tuân theo các bước sau đây :
1. Đảm bảo nhiệt độ từ 25 - 30C bằng cách che mát hoặc thắp đèn sưởi ấm tùy thuộc vào môi trường.
2. Duy trì độ ẩm 70-80% để ngăn chặn sự héo úa của lá và suy giảm sức khỏe tổng thể.
3. Tưới nước thường xuyên, đặc biệt là trong thời kỳ khô hạn, ví dụ như mùa hè. Nhà kính đòi hỏi lượng nước lớn hơn so với ngoài trời do tốc độ bay hơi cao hơn. Có thể tưới mỗi ngày hai đến ba lần đối với nhà kính và năm lần đối với bên ngoài.
4. Lan Vanda thích ánh sáng mạnh và sẽ nở hoa dưới cường độ chiếu sáng cao. Đặt chúng gần cửa sổ hướng Nam hoặc sử dụng hệ thống chiếu sáng nhân tạo chuyên dùng cho cây cối.
5. Bón phân cân đối hàng tuần trong suốt cả năm vì loài lan này có nhu cầu dinh dưỡng cao. Sử dụng loại phân bón tan chậm giải phóng chất dinh dưỡng đều đặn.
6. Thay chậu vài năm một lần để cung cấp đất mới và kích thước chậu phù hợp với rễ đang phát triển. Chọn hỗn hợp bầu gồm vỏ cây, rêu than bùn và đá trân châu để thoát nướ

/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

phòng trừ côn trùng và nấm bệnh cho cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây. </s>
 Để trồng và chăm sóc cây lan chu đinh, hãy làm theo các bước sau đây:
1. Chọn loại đất phù hợp (đất thịt, đất mùn hoặc đất cát) với độ tơi xốp, khả năng thoát nước và giữ ẩm cao.
2. Trồng cây ở nơi có ánh sáng mặt trời đầy đủ nhưng tránh nắng trực tiếp.
3. Tưới nước thường xuyên để duy trì độ ẩm cho đất.
4. Bón phân cân đối mỗi tháng một lần trong mùa sinh trưởng.
5. Cắt tỉa những cành chết hoặc hư hỏng để thúc đẩy sự phát triển mới.
6. Nhân giống bằng phương pháp tách bụi, chọn bụi đã lớn và khỏe mạnh rồi dùng dao sắc cắt rời phần thân bên ngoài.
7. Gieo hạt xuống đất sâu khoảng 0,5 cm và tưới nhẹ nhàng.
8. Che phủ khu vực gieo hạt bằng vật liệu bảo vệ khỏi mưa quá nhiều.
9. Khi cây con mọc lên thì di dời chúng sang vị trí mong muốn.
Bằng 

/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/trung/miniconda3/envs/dev/lib/pyth

phòng trừ côn trùng và nấm bệnh cho cây sầu riêng, sử dụng các sản phẩm sinh học và chất phòng trừ côn trùng/nấm bệnh định kỳ. Việc chăm sóc sầu riêng trong giai đoạn nuôi trái rất quan trọng đối với sức khỏe tổng thể và sự phát triển của cây. </s>
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
Cannot find document revelance
 Vệ sinh gốc sầu riêng bằng cách quét dọn, thu gom lá cây và làm sạch cỏ xung quanh gốc cây; tỉa cành để tạo điều kiện thông thoáng và giảm nguy cơ bị tấn công bởi sâu bệnh; bón phân cân đối NPK với tỷ lệ 1-2-3 hoặc 1-1,5-2,5 để giúp cây nhanh chóng phục hồi năng lượng dự trữ trong giai đoạn mang hoa, đậu quả trước đó. Ngoài ra, sử dụng thuốc diệt nấm như Benomyl, Ridomil Gold

In [42]:
print("__" * 100)

________________________________________________________________________________________________________________________________________________________________________________________________________
